In [1]:
from sklearn import svm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import preprocessing
import pandas as pd
import numpy as np

In [2]:
import sys
import os

In [6]:
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
sys.path

['/home/david/miniconda2/envs/t10/lib/python36.zip',
 '/home/david/miniconda2/envs/t10/lib/python3.6',
 '/home/david/miniconda2/envs/t10/lib/python3.6/lib-dynload',
 '',
 '/home/david/miniconda2/envs/t10/lib/python3.6/site-packages',
 '/home/david/miniconda2/envs/t10/lib/python3.6/site-packages/IPython/extensions',
 '/home/david/.ipython',
 '/home/david/Documents/proj/tag_recommendation/base',
 '/home/david/Documents/proj/tag_recommendation']

In [8]:
import lib.notebook_helpers as nh

Using TensorFlow backend.


## Filter out tags

In [14]:
import pickle

In [15]:
with open("../../data/dev_tag_dataset.pkl", 'rb') as f:
        X, y, mlbx, mlby = pickle.load(f)

Now the quality of the dataset should be good enough.

In [10]:
nh.tag_stats(dataset)

NameError: name 'dataset' is not defined

In [11]:
# utility line for comparing full sets of tags
# dataset.User_tags.map(lambda x: set(x))[dataset.User_tags.map(lambda x: set(x)) == {'highway', 'road', 'route', 'shield', 'sign', 'virginia'}]

In [194]:
dataset = nh.generate_data()

In [195]:
dataset.shape

(17427, 2)

In [197]:
nh.tag_stats(dataset)

number of autotags:
total: 239409
unique: 1000
ratio autotags/item: 13.737820623170942

number of user tags:
total: 150547
unique: 986
ratio user_tags/item: 8.638721524071842


In [179]:
data_dir = "/home/david/Documents/proj/tag-rec/src/notebooks/flickr100m/results"
#data_dir = "/Users/davidmalmstrom/Mounts/tag-rec/src/notebooks/flickr100m/results"

dataset = pd.read_pickle(data_dir + "/" + "preprocessed_user_auto_tags.pkl")

In [180]:
all_user_tags = pd.Series([tag for liste in dataset.User_tags for tag in liste]).value_counts()

In [181]:
all_user_tags.shape

(2015,)

## Reshape dataset

We need the classes (User_tags) to be one per sample, and the features (autotags) to be one-hot-encoded.

Starting with the one-hot-encoding:

In [5]:
mlb = MultiLabelBinarizer()
one_hot_autotags = pd.DataFrame(mlb.fit_transform(dataset.autotags), columns=mlb.classes_, index=dataset.index)

In [6]:
one_hot_autotags.sample(3)

,aardvark,abbey,abstract,abutment,accordion,acropolis,adapter,adobe,aerial,african daisy,...,writing,yacht,yak,yard,yarn,yearling,yellow,yorkshire terrier,yurt,zebra
Line_number,,,,,,,,,,,,,,,,,,,,,
233928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Skip unnesting:



In [7]:
X = one_hot_autotags

In [8]:
mlb2 = MultiLabelBinarizer()
y = mlb2.fit_transform(dataset.User_tags)

In [11]:
dataset = nh.generate_data(10000)

In [12]:
X, y, mlbx, mlby = nh.reshape_data(dataset)

## Split into training test set

In [16]:
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.metrics import jaccard_similarity_score, accuracy_score, f1_score, precision_score, recall_score

In [14]:
steps = [('tfidf', TfidfTransformer()), ('clf', OneVsRestClassifier(LinearSVC()))]
pipeline = Pipeline(steps)

In [157]:
parameters = {'clf__estimator__C':[0.001,0.1,1,10,100,10e5]}

In [166]:
grid = GridSearchCV(pipeline, param_grid=parameters, scoring=make_scorer(jaccard_similarity_score), return_train_score=True)

In [159]:
grid.fit(x_train, y_train)

/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/svm/base

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__estimator__C': [0.001, 0.1, 1, 10, 100, 1000000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(jaccard_similarity_score), verbose=0)

In [167]:
pd.DataFrame(grid.cv_results_)

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [17]:
#clf = OneVsRestClassifier(MultinomialNB())
clf = OneVsRestClassifier(LinearSVC(C=10))
#clf = RandomForestClassifier(n_estimators=100, max_depth=100)

In [20]:
x_test

<5400x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 73195 stored elements in Compressed Sparse Row format>

In [21]:
clf.fit(x_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(C=10, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=1000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0),
                    n_jobs=None)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
transformer = TfidfTransformer()
x_train = transformer.fit_transform(x_train)
x_test = transformer.transform(x_test)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

y_pred_prob = clf.predict_proba(x_test)
tops = nh.get_top_n_tags(y_pred_prob)
new_preds = nh.from_keras_format(list(map(lambda x: x + 1, tops)), y.shape[1])

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [220]:
importlib.reload(nh)


<module 'lib.notebook_helpers' from '/home/david/Documents/proj/tag-rec/src/lib/notebook_helpers.py'>

In [221]:
lin_svc = nh.ModelWrapper(OneVsRestClassifier(LinearSVC(C=10)), X, y, mlbx, mlby, tfidf=True)

In [222]:
lin_svc.print_scores()

jaccard score: 0.04934387799133868
accuracy score: 0.0
f1-score: 0.08348689598689597
recall-score: 0.0629681105706123
precision-score: 0.12384206529992407
Hamming loss: 0.012106152805949967
Hamming score: 0.04934387799133868
---


In [233]:
lin_svc.random_check()

('car', 'dog', 'dogs', 'fire', 'holiday', 'horse', 'ohio', 'parade', 'police', 'racing', 'santa', 'truck')
('italian', 'scotland')
index: 1483


In [234]:
naive_bayes = nh.ModelWrapper(OneVsRestClassifier(MultinomialNB()), X, y, mlbx, mlby, tfidf=False, has_probs=True)

In [236]:
naive_bayes.print_scores()

jaccard score: 0.083870305860422
accuracy score: 0.0
f1-score: 0.10760884307608842
recall-score: 0.3075909126153369
precision-score: 0.06521131622639029
Hamming loss: 0.0444868154158215
Hamming score: 0.083870305860422
---


In [242]:
naive_bayes.print_scores(top_n=True)

jaccard score: 0.10798507766911292
accuracy score: 0.0
f1-score: 0.16904446732362602
recall-score: 0.1336744979452586
precision-score: 0.22986666666666666
Hamming loss: 0.011459770114942529
Hamming score: 0.10798507766911292
---


In [248]:
naive_bayes.random_check(top_n=True)

('architecture', 'building', 'flowers', 'las', 'statue', 'vegas', 'waterfall')
('architecture', 'building', 'city', 'downtown', 'skyscraper')
index: 1543


In [78]:
max(y_pred_prob[8])

0.9995550641270531

In [69]:
# new bayes
nh.print_all_scores(y_test, new_preds)

jaccard score: 0.10406595965705501
accuracy score: 0.0
f1-score: 0.1642326793529036
recall-score: 0.1300927007867882
precision-score: 0.22266666666666668
Hamming loss: 0.011492224475997296
Hamming score: 0.10406595965705501
---


In [96]:
# bayes:
nh.print_all_scores(y_test, y_pred)

jaccard score: 0.08253440740271828
accuracy score: 0.0
f1-score: 0.10956451747203641
recall-score: 0.3075095427280517
precision-score: 0.06665709798888908
Hamming loss: 0.043382691007437454
Hamming score: 0.08253440740271828
---


In [94]:
print_all_scores(y_test, y_pred)


NameError: name 'print_all_scores' is not defined

In [24]:
print("jaccard score: " + str(jaccard_similarity_score(y_test, y_pred)))
print("accuracy score: " + str(accuracy_score(y_test, y_pred)))
print("f1-score: " + str(f1_score(y_test, y_pred, average='weighted')))
print_score(y_pred)

jaccard score: 0.10815783766338541
accuracy score: 0.0013333333333333333
f1-score: 0.1462990799197635
Hamming loss: 0.04279871664978048
Hamming score: 0.10815783766338541
---


In [221]:
from random import randint

index = randint(0, len(y_test))
print(mlby.inverse_transform(y_test)[index])
print(mlby.inverse_transform(y_pred)[index])

('auto', 'car', 'china', 'ford', 'race', 'racing', 'stage')
('antique', 'auto', 'automobile', 'big', 'bike', 'bmw', 'ca', 'car', 'car+show', 'cars', 'classic', 'club', 'course', 'day', 'engine', 'fast', 'ford', 'foto', 'historic', 'honda', 'hot', 'march', 'may', 'motor', 'paper', 'park', 'photo', 'photos', 'pictures', 'power', 'race', 'racing', 'rally', 'reflections', 'show', 'speed', 'sport', 'sports', 'track', 'vehicle', 'vintage', 'wild')


In [93]:
nh.random_check(y_test, new_preds, mlbx, mlby)

('barcelona', 'catalunya', 'coast', 'hiking', 'october', 'spain', 'travel')
('flower', 'flowers', 'green', 'nature', 'trees')
index: 2281


In [54]:
y_pred_svc = y_pred

In [72]:
np.mean(np.sum(y_pred, axis=1))

42.715

In [73]:
np.mean(np.sum(y_pred_svc, axis=1))

5.01

In [178]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Keras:

In [232]:
print("X-shape: " + str(X.shape))
print("y-shape: " + str(y.shape))

X-shape: (10000, 1000)
y-shape: (10000, 988)


In [ ]:
n_samples = 8000
X = X.head(n_samples)

y = y[:n_samples]

In [510]:
X_we = np.zeros(X.shape)

In [520]:
X_keras = to_keras_format(X)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=9000)

In [68]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_sample_weight(class_weight='balanced', y=y)

In [67]:
weights.shape

(8000,)

In [176]:
X, Y = make_multilabel_classification(n_samples=8000, length=50, n_features=1000, n_classes=15, n_labels=5,
                                      allow_unlabeled=True,
                                      random_state=3)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [16]:
from sklearn.datasets import make_multilabel_classification
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.metrics import jaccard_similarity_score
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.optimizers import SGD
from keras.metrics import top_k_categorical_accuracy

In [329]:
max_length = max([len(row) for row in x_train])
x_train = pad_sequences(x_train, maxlen=max_length, padding='post')

In [333]:
max_length = max([len(row) for row in x_test])
x_test = pad_sequences(x_test, maxlen=max_length, padding='post')

## Newest

In [682]:
y_train.shape[1]

987

In [17]:
model = Sequential()
model.add(Dense(950, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0015), metrics=['accuracy', nh.mean_iou, top_k_categorical_accuracy])

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple.h5', save_best_only=True)
]

In [18]:
model.fit(x_train, y_train, epochs=7, batch_size=35, callbacks=callbacks, validation_split=0.1)

preds_orgs = model.predict(x_test)
preds[preds_orgs>=0.5] = 1
preds[preds_orgs<0.5] = 0

Train on 7200 samples, validate on 800 samples
Epoch 1/7
7200/7200 [==============================] - 31s 4ms/step - loss: 0.0765 - acc: 0.9874 - mean_iou: 0.9598 - top_k_categorical_accuracy: 0.0861 - val_loss: 0.0468 - val_acc: 0.9911 - val_mean_iou: 0.9581 - val_top_k_categorical_accuracy: 0.1800
Epoch 2/7
7200/7200 [==============================] - 15s 2ms/step - loss: 0.0435 - acc: 0.9913 - mean_iou: 0.9538 - top_k_categorical_accuracy: 0.1508 - val_loss: 0.0437 - val_acc: 0.9912 - val_mean_iou: 0.9527 - val_top_k_categorical_accuracy: 0.1650
Epoch 3/7
7200/7200 [==============================] - 15s 2ms/step - loss: 0.0404 - acc: 0.9913 - mean_iou: 0.9507 - top_k_categorical_accuracy: 0.1738 - val_loss: 0.0427 - val_acc: 0.9912 - val_mean_iou: 0.9507 - val_top_k_categorical_accuracy: 0.1988
Epoch 4/7
7200/7200 [==============================] - 15s 2ms/step - loss: 0.0384 - acc: 0.9914 - mean_iou: 0.9493 - top_k_categorical_accuracy: 0.1881 - val_loss: 0.0423 - val_acc: 0.9912 -

NameError: name 'preds' is not defined

In [755]:
preds_orgs = model.predict(x_test)


In [255]:
preds = preds_orgs.copy()
preds[preds>=0.14] = 1
preds[preds<0.14] = 0

In [777]:
np.sum(preds, axis=1)

array([ 3., 11., 16., ..., 12., 15.,  5.], dtype=float32)

In [19]:
tops = nh.get_top_n_tags(preds_orgs)

In [23]:
new_preds = nh.from_keras_format(list(map(lambda x: x + 1, tops)), y.shape[1])

In [24]:
nh.print_all_scores(y_test, new_preds)

jaccard score: 0.1055546644792059
accuracy score: 0.0
f1-score: 0.1636773787951629
recall-score: 0.1298632709348043
precision-score: 0.2213
Hamming loss: 0.011468052738336714
Hamming score: 0.1055546644792059
---


In [801]:
np.sum(preds, axis=1)

array([ 3., 11., 16., ..., 12., 15.,  5.], dtype=float32)

In [799]:
np.sum(y_test, axis=1)

array([ 7,  7,  7, ...,  7, 14,  6])

In [55]:
#sum(preds[1827])
pred_test = preds_orgs.copy()

In [56]:
pred_test[pred_test<0.1] = 0

In [76]:
pred_test[1347]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [291]:
nh.random_check(y_test, preds, mlbx, mlby)

('bar', 'buildings', 'bus', 'nippon', 'old', 'wave')
('bands', 'buildings', 'bus', 'life', 'new+jersey', 'wa', 'washington+dc', 'wave')
index: 619


In [807]:
y_test != preds

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [811]:
y_test.shape

(2000, 987)

In [813]:
y_train.shape

(8000, 987)

In [818]:
np.sum(np.all(y_test == y_train[6000:8000], axis=1))

2

## Oldest

In [525]:
X.shape[1]

1000

In [524]:
x_train.shape[1]

77

In [652]:
model = Sequential()
model.add(Embedding(input_dim=X.shape[1] + 1, output_dim=20, mask_zero=False, input_length=x_train.shape[1]))
model.add(Dropout(0.15))
#model.add(MeanPool(output_dim=x_train.shape[1]))
model.add(GlobalMaxPool1D())
model.add(Dense(units=y.shape[1], activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_73 (Embedding)     (None, 77, 20)            20020     
_________________________________________________________________
dropout_90 (Dropout)         (None, 77, 20)            0         
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 20)                0         
_________________________________________________________________
dense_89 (Dense)             (None, 987)               20727     
Total params: 40,747
Trainable params: 40,747
Non-trainable params: 0
_________________________________________________________________


In [654]:
model = Sequential()
model.add(Embedding(input_dim=X.shape[1] + 1, output_dim=64, mask_zero=False, input_length=x_train.shape[1]))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(units=y.shape[1], activation='sigmoid'))

model.compile(optimizer=Adam(0.015), loss='binary_crossentropy', metrics=['accuracy', mean_iou])
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple.h5', save_best_only=True)
]

history = model.fit(x_train, y_train,
                    class_weight=None,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 7200 samples, validate on 800 samples
Epoch 1/20
7200/7200 [==============================] - 91s 13ms/step - loss: 0.0700 - acc: 0.9890 - mean_iou: 0.9688 - val_loss: 0.0497 - val_acc: 0.9906 - val_mean_iou: 0.9680
Epoch 2/20
7200/7200 [==============================] - 11s 2ms/step - loss: 0.0440 - acc: 0.9913 - mean_iou: 0.9620 - val_loss: 0.0460 - val_acc: 0.9908 - val_mean_iou: 0.9585
Epoch 3/20
7200/7200 [==============================] - 12s 2ms/step - loss: 0.0416 - acc: 0.9914 - mean_iou: 0.9575 - val_loss: 0.0446 - val_acc: 0.9908 - val_mean_iou: 0.9566
Epoch 4/20
7200/7200 [==============================] - 11s 2ms/step - loss: 0.0405 - acc: 0.9914 - mean_iou: 0.9551 - val_loss: 0.0441 - val_acc: 0.9908 - val_mean_iou: 0.9568
Epoch 5/20
7200/7200 [==============================] - 11s 2ms/step - loss: 0.0396 - acc: 0.9915 - mean_iou: 0.9536 - val_loss: 0.0443 - val_acc: 0.9908 - val_mean_iou: 0.9575
Epoch 6/20
7200/7200 [==============================] - 12s 2ms/ste

In [301]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[18 29  0  0]
 [38 17  0  0]
 [16 44  0  0]
 [ 8 17  0  0]
 [42  0  0  0]
 [ 3  0  0  0]
 [25 44  0  0]
 [39 38  0  0]
 [25 17  0  0]
 [ 7 46 29 10]]


In [482]:
keras.metrics.binary_accuracy(labels, predLabels)
keras.metrics.categorical_accuracy(labels, predLabels)
keras.metrics.sparse_categorical_accuracy(labels, predLabels)

TypeError: Input 'y' of 'Equal' Op has type float64 that does not match type int64 of argument 'x'.

In [655]:
simple_model = load_model('model-simple.h5', custom_objects={'mean_iou': mean_iou})
metrics = simple_model.evaluate(x_test, y_test)
print("{}: {}".format(simple_model.metrics_names[0], metrics[0]))
print("{}: {}".format(simple_model.metrics_names[1], metrics[1]))

2000/2000 [==============================] - 30s 15ms/step
loss: 0.043100813120603564
acc: 0.9912857160568237


In [656]:
asd = simple_model.predict(x_test, verbose=1)
qwe = simple_model.predict_classes(x_test, verbose=1)

2000/2000 [==============================] - 0s 35us/step


In [337]:
qwe

array([593, 439, 768, ..., 849, 601, 367])

In [343]:
asd[0][593]

0.19119477

In [657]:
test = from_keras_format(get_top_n_tags(asd), num_classes=y.shape[1])

In [658]:
print("jaccard score: " + str(jaccard_similarity_score(y_test, test)))
print("accuracy score: " + str(accuracy_score(y_test, test)))
print("f1-score: " + str(f1_score(y_test, test, average='weighted')))
print_score(test)

jaccard score: 0.013118153600187069
accuracy score: 0.0
f1-score: 0.016478487646567997
Hamming loss: 0.013664133738601824
Hamming score: 0.013118153600187069
---


/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/davidmalmstrom/.virtualenvs/proj/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [662]:
from random import randint

index = randint(0, len(test))
print(mlby.inverse_transform(y_test)[index])
print(mlby.inverse_transform(test)[index])

('boat', 'costumes', 'drinking', 'fun', 'girls', 'hot', 'marathon', 'party', 'runners', 'running', 'san+francisco', 'street')
('forest+city+ratner', 'jovem', 'novembro', 'nr1', 'speed')


In [159]:
filter_length = 300

model = Sequential()
model.add(Embedding(input_dim=X.shape[1], output_dim=20))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(units=y.shape[1]))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, None, 20)          20000     
_________________________________________________________________
dropout_22 (Dropout)         (None, None, 20)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 300)         18300     
_________________________________________________________________
global_max_pooling1d_21 (Glo (None, 300)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 998)               300398    
_________________________________________________________________
activation_2 (Activation)    (None, 998)               0         
Total params: 338,698
Trainable params: 338,698
Non-trainable params: 0
_________________________________________________________________
Trai

KeyboardInterrupt: 

In [134]:
x_test.shape

(1600, 1518)

In [138]:
asd.shape

(1600, 998)

In [147]:
asd

array([[0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021],
       [0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021],
       [0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021],
       ...,
       [0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021],
       [0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021],
       [0.01121756, 0.02389684, 0.0029079 , ..., 0.01037702, 0.0070821 ,
        0.01716021]], dtype=float32)

In [142]:
set(qwe)

{624}

In [ ]:
1+1